In [1]:
!pip install pandas scikit-learn datasets accelerate

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import sys


In [3]:
print(sys.executable)

C:\Users\Educational Purpose\AppData\Local\Programs\Python\Python313\python.exe


In [4]:
df = pd.read_csv(r"C:\Users\Educational Purpose\Downloads\posts_train_paraphrased_full.csv") 

In [5]:
df.head

<bound method NDFrame.head of                                          ID  \
0      691324c4-5c30-44e0-b9e4-45b4f0715e21   
1      d4295391-9ca5-4398-b7c8-687e4a984ef1   
2      58937fa5-3c2c-426b-8255-5a140fbab675   
3      7daf364c-3b33-4cbe-be37-a214edf9a73e   
4      22518271-4bb4-4caf-b683-7305da519288   
...                                     ...   
13724  386a4117-0324-4b25-8330-b1a6857ccaa6   
13725  4dd92d63-1231-410d-b049-5d8c430c2f36   
13726  dbde1d3e-e527-4cf8-8045-2093aec2a784   
13727  5355fad6-5bb4-4092-8428-9b8527a6d261   
13728  4b3f878d-9398-452a-992d-d73c28f0c849   

                                                    post  class_name class_id  \
0      i was making questions for my students and i r...        none        5   
1      i've recently requested testing accommodations...        adhd        0   
2      **cambodia** * koh rong: amazing beaches and a...        none        5   
3      synesthesia. what is synesthesia? according to...        none        5   


In [6]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [7]:
df.head

<bound method NDFrame.head of                                          ID  \
0      691324c4-5c30-44e0-b9e4-45b4f0715e21   
1      d4295391-9ca5-4398-b7c8-687e4a984ef1   
2      58937fa5-3c2c-426b-8255-5a140fbab675   
3      7daf364c-3b33-4cbe-be37-a214edf9a73e   
4      22518271-4bb4-4caf-b683-7305da519288   
...                                     ...   
13724  386a4117-0324-4b25-8330-b1a6857ccaa6   
13725  4dd92d63-1231-410d-b049-5d8c430c2f36   
13726  dbde1d3e-e527-4cf8-8045-2093aec2a784   
13727  5355fad6-5bb4-4092-8428-9b8527a6d261   
13728  4b3f878d-9398-452a-992d-d73c28f0c849   

                                                    post  class_name class_id  \
0      i was making questions for my students and i r...        none        5   
1      i've recently requested testing accommodations...        adhd        0   
2      **cambodia** * koh rong: amazing beaches and a...        none        5   
3      synesthesia. what is synesthesia? according to...        none        5   


In [8]:
df.columns = df.columns.str.strip()


In [9]:
print(df.columns)


Index(['ID', 'post', 'class_name', 'class_id', 'post_paraphrased'], dtype='object')


In [10]:
df = df[["post", "post_paraphrased", "class_name"]].dropna()

In [11]:
df = df.groupby("class_name").filter(lambda x: len(x) > 1)

In [12]:
train_df, temp_df = train_test_split(
    df, test_size=0.2, stratify=df["class_name"], random_state=42
)
val_df, test_df = train_test_split(
    temp_df, test_size=0.5, stratify=temp_df["class_name"], random_state=42
)


In [13]:
train_combined = pd.concat([
    train_df[["post", "class_name"]].rename(columns={"post": "text"}),
    train_df[["post_paraphrased", "class_name"]].rename(columns={"post_paraphrased": "text"})
])

In [14]:
val_texts = val_df["post"].tolist()
val_labels = val_df["class_name"].tolist()


In [15]:
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train = vectorizer.fit_transform(train_combined["text"])
X_val = vectorizer.transform(val_texts)

In [16]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, train_combined["class_name"])

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [17]:
preds = model.predict(X_val)
f1 = f1_score(val_labels, preds, average="weighted")

In [24]:
print(f" Weighted F1 Score (80% original + 80% paraphrased): {f1:.4f}")

 Weighted F1 Score (80% original + 80% paraphrased): 0.7489
